<a href="https://colab.research.google.com/github/Wupadova/Machine-and-Deep-Learning-Mod.-A-/blob/main/ML_project_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from urllib.request import urlopen

def load_fashion_mnist():

    url_base = "https://www.math.unipd.it/~dasan/"
    Y_train = np.frombuffer(urlopen(url_base + "train-labels-idx1-ubyte").read(), dtype=np.uint8, offset=8)
    X_train = np.frombuffer(urlopen(url_base + "train-images-idx3-ubyte").read(), dtype=np.uint8, offset=16).reshape(len(Y_train), 784) # besides loadng 
                                                                                            #the data, I already flatten it into a vector
    Y_test = np.frombuffer(urlopen(url_base + "t10k-labels-idx1-ubyte").read(), dtype=np.uint8, offset=8)
    X_test = np.frombuffer(urlopen(url_base + "t10k-images-idx3-ubyte").read(), dtype=np.uint8, offset=16).reshape(len(Y_test), 784)

    return X_train, Y_train, X_test, Y_test

In [ ]:
X_train, Y_train, X_test, Y_test = load_fashion_mnist() # you can use this function (copying the whole box) in your code to easily load the data
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(60000, 784) (60000,) (10000, 784) (10000,)


In [ ]:
# split the train and validation sets
from sklearn.model_selection import train_test_split
X_train_, X_val, y_train_, y_val = train_test_split(X_train, Y_train, test_size=0.2)

In [ ]:
# check if the data set is split correctly
print(X_train_.shape, X_val.shape, X_test.shape)

(48000, 784) (12000, 784) (10000, 784)


In [ ]:
# check the output
print(len(set(Y_train)))
print(set(Y_train))

10
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [ ]:
print(X_train[0:5, 0:5])

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [ ]:
print(Y_train[0:5])

[9 0 0 3 0]


In [ ]:
# try with different models: decision tree, random forest, SVC and in the end NN starting with raw data
# import all the possibly necessary libraries 
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import tensorflow #the backend used by Keras (there are different beckend)
from tensorflow.keras.models import Sequential #import the type of mpdel: sequential (e.g., MLP)
from tensorflow.keras.layers import Input, Dense #simple linear layer
from tensorflow.keras.utils import to_categorical # transformation for classification labels
from keras.utils.vis_utils import plot_model
from tensorflow.python.framework.random_seed import set_random_seed
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
import joblib
from sklearn.decomposition import PCA
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier

## Try the models with raw data

In [ ]:
# decision tree 

dt_grid_param = {
    'criterion' : ['entropy'],
    'max_depth' : (None, 3, 5, 7)   
}

dt_clf = DecisionTreeClassifier(random_state=123)
dt_v1 = GridSearchCV(dt_clf, dt_grid_param, n_jobs=-1, cv=5)
dt_v1.fit(X_train_, y_train_)

# random forest 

rf_grid_param = {
    'n_estimators' : (5, 10, 20),
    'criterion' : ['entropy'],
    'max_depth' : (None, 3, 5, 7)
}

rf_clf = RandomForestClassifier(random_state=123)
rf_v1 = GridSearchCV(rf_clf, rf_grid_param, n_jobs=-1, cv=5)
rf_v1.fit(X_train_, y_train_)



In [ ]:
# try with knn classifier
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()

knn_grid_param = {
    'n_neighbors' : (5, 10, 20),
    'weights' : ('uniform', 'distance')
}

knn_v1 = GridSearchCV(knn_clf, knn_grid_param, n_jobs=-1, cv=5)
knn_v1.fit(X_train_, y_train_)

In [ ]:
# extra-tree classifier 
from sklearn.tree import ExtraTreeClassifier

ext_tree_clf = ExtraTreeClassifier(random_state=123)

ext_tree_param = {
    'criterion' : ['entropy'],
    'splitter' : ['random', 'best'],
}

ext_tree_v1 = GridSearchCV(ext_tree_clf, ext_tree_param, n_jobs=-1, cv=5)
ext_tree_v1.fit(X_train_, y_train_)

In [ ]:
# try with OVO and OVR strategies
# OVO and OVR for perceptron
model = Perceptron(fit_intercept=True, n_jobs=-1, random_state=123)

# define OVO strategy and train
ovo = OneVsOneClassifier(model)
ovo.fit(X_train_, y_train_)

# define OVR strategy and train
ovr = OneVsRestClassifier(model)
ovr.fit(X_train_, y_train_)

In [ ]:
# NN 
# define the early stopping
es = EarlyStopping(monitor='val_loss', #quantity to be monitored
                   mode='min', #we look for decreasing patterns stop 
                   patience = 3, #number of epochs with no improvement
                   verbose=1)

# define the NN settings
feature_vector_length = len(X_train_[0])  # assign the number of features
num_classes = len(set(Y_train))   # assign the number of the classes
print(feature_vector_length, num_classes)  # check the result

# convert the ground truth from numerical to categorical representation
y_train_cat = to_categorical(y_train_, num_classes)
y_val_cat = to_categorical(y_val, num_classes)

# set the seed
np.random.seed(123)
set_random_seed(2)

# define the NN model
model = Sequential()   # define how the 'model' looks like
model.add(Dense(input_dim=feature_vector_length, units=feature_vector_length, activation='relu'))  # input layer
model.add(Dense(num_classes, activation='softmax'))  # output layer

# configure the model and start training 
model.compile(loss='categorical_crossentropy',   # loss metric
             optimizer='sgd',   # optimizer
             metrics=['accuracy'])  # displayed metric

history = model.fit(X_train_, y_train_cat, epochs=500, batch_size=32, verbose=1, validation_split=0.1, callbacks=[es])

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
# plt.ylim(0.8, 1)
plt.show()


# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# check the result of the above two models (original dataset is too large and too complicated
# for the SVM, thus we will perform a pre-processing later for the SVM)
# decision
y_train_pred = dt_v1.predict(X_train_)
y_val_pred = dt_v1.predict(X_val)
print(f"Decision Tree.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

# random forest
y_train_pred = rf_v1.predict(X_train_)
y_val_pred = rf_v1.predict(X_val)
print(f"Random Forest.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

# knn evaluation
y_train_pred = knn_v1.predict(X_train_)
y_val_pred = knn_v1.predict(X_val)
print(f"KNN.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

# extra tree evaluation
y_train_pred = ext_tree_v1.predict(X_train_)
y_val_pred = ext_tree_v1.predict(X_val)
print(f"Extra Tree.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

# ovo perceptron evaluation
y_train_pred = ovo.predict(X_train_)
y_val_pred = ovo.predict(X_val)
print(f"OVO_perceptron.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

# ovr perceptron evaluation
y_train_pred = ovr.predict(X_train_)
y_val_pred = ovr.predict(X_val)
print(f"OVR_perceptron.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

In [ ]:
# show the best hpyer parameters
print('Decison Tree:', dt_v1.best_params_)
print('Random Forest:', rf_v1.best_params_)
print('Extra Tree:', ext_tree_v1.best_params_)
print('KNN:', knn_v1.best_params_)

## Some models are overfitted, we check the data to see if they are balanced

In [ ]:
# first we check the Y dataset
for a in set(y_train_):
    result = 0
    for b in range(len(y_train_)):
        if y_train_[b] == a:
            result += 1
    print(result)


## The training set is well balanced, thus we can try to perform the pre-processing

In [ ]:
# try with MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

# define the scaler
scaler = MinMaxScaler()

scaler.fit(X_train_)   # fit the scaler with the training set

# transform the dataset
X_train_scaled = scaler.transform(X_train_)  # scaled training set
X_val_scaled = scaler.transform(X_val)  # scaled validation set

In [ ]:
# train the models with the scaled data
# decision tree
dt_v1.fit(X_train_scaled, y_train_)

# random forest 
rf_v1.fit(X_train_scaled, y_train_)

# extra tree
ext_tree_v1.fit(X_train_scaled, y_train_)

# knn
knn_v1.fit(X_train_scaled, y_train_)

# OVO strategy for perceptron
ovo.fit(X_train_scaled, y_train_)

# OVR strategy for perceptron
ovr.fit(X_train_scaled, y_train_)

In [ ]:
# evaluate the result
# decision
y_train_pred = dt_v1.predict(X_train_scaled)
y_val_pred = dt_v1.predict(X_val_scaled)
print(f"Decision Tree.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

# random forest
y_train_pred = rf_v1.predict(X_train_scaled)
y_val_pred = rf_v1.predict(X_val_scaled)
print(f"Random Forest.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

# knn evaluation
y_train_pred = knn_v1.predict(X_train_scaled)
y_val_pred = knn_v1.predict(X_val_scaled)
print(f"KNN.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

# extra tree evaluation
y_train_pred = ext_tree_v1.predict(X_train_scaled)
y_val_pred = ext_tree_v1.predict(X_val_scaled)
print(f"Extra Tree.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

# ovo perceptron evaluation
y_train_pred = ovo.predict(X_train_scaled)
y_val_pred = ovo.predict(X_val_scaled)
print(f"OVO_perceptron.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

# ovr perceptron evaluation
y_train_pred = ovr.predict(X_train_scaled)
y_val_pred = ovr.predict(X_val_scaled)
print(f"OVR_perceptron.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

In [ ]:
# show the best hpyer parameters
print('Decison Tree:', dt_v1.best_params_)
print('Random Forest:', rf_v1.best_params_)
print('Extra Tree:', ext_tree_v1.best_params_)
print('KNN:', knn_v1.best_params_)

In [ ]:
# NN
history = model.fit(X_train_scaled, y_train_cat, epochs=500, batch_size=32, verbose=1, validation_split=0.1, callbacks=[es])

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
# plt.ylim(0.8, 1)
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## Now, we can see that the NN model improved a lot, however, other models are less 'sensitive' to the scaler.  Now, we can try the SVC and logistic regression but the PCA is needed beforehand.

In [ ]:
# define the PCA and set the number of components
pca = PCA(n_components=0.95, random_state=123)

# fit and transform the data
pca.fit(X_train_scaled)   # fit the pca with scaled training set
X_train_reduced = pca.transform(X_train_scaled)   # transform the training set
X_val_reduced = pca.transform(X_val_scaled)  # transform the validation set

# check the reduced shape of our data
print(X_train_reduced.shape)
print(X_val_reduced.shape)

## Now, we can try all the models again with the reduced data and see the results.

In [ ]:
# train the models with the reduced data
# decision tree
dt_v1.fit(X_train_reduced, y_train_)

# random forest 
rf_v1.fit(X_train_reduced, y_train_)

# extra tree
ext_tree_v1.fit(X_train_reduced, y_train_)

# knn
knn_v1.fit(X_train_reduced, y_train_)

# OVO strategy for perceptron
ovo.fit(X_train_reduced, y_train_)

# OVR strategy for perceptron
ovr.fit(X_train_reduced, y_train_)

# OVO and OVR strategies for logistic regression
model_lr = LogisticRegression(fit_intercept=True, n_jobs=-1, random_state=123)

# define OVO strategy and train
ovo_lr = OneVsOneClassifier(model_lr)
ovo_lr.fit(X_train_reduced, y_train_)

# define OVR strategy and train
ovr_lr = OneVsRestClassifier(model_lr)
ovr_lr.fit(X_train_reduced, y_train_)

In [ ]:
# evaluate the result
# decision
y_train_pred = dt_v1.predict(X_train_reduced)
y_val_pred = dt_v1.predict(X_val_reduced)
print(f"Decision Tree.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

# random forest
y_train_pred = rf_v1.predict(X_train_reduced)
y_val_pred = rf_v1.predict(X_val_reduced)
print(f"Random Forest.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

# knn evaluation
y_train_pred = knn_v1.predict(X_train_reduced)
y_val_pred = knn_v1.predict(X_val_reduced)
print(f"KNN.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

# extra tree evaluation
y_train_pred = ext_tree_v1.predict(X_train_reduced)
y_val_pred = ext_tree_v1.predict(X_val_reduced)
print(f"Extra Tree.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

# ovo perceptron evaluation
y_train_pred = ovo.predict(X_train_reduced)
y_val_pred = ovo.predict(X_val_reduced)
print(f"OVO_perceptron.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

# ovr perceptron evaluation
y_train_pred = ovr.predict(X_train_reduced)
y_val_pred = ovr.predict(X_val_reduced)
print(f"OVR_perceptron.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

# ovo logistic regression evaluation
y_train_pred = ovo_lr.predict(X_train_reduced)
y_val_pred = ovo_lr.predict(X_val_reduced)
print(f"OVO_logistic regression.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

# ovr logistic regression evaluation
y_train_pred = ovr_lr.predict(X_train_reduced)
y_val_pred = ovr_lr.predict(X_val_reduced)
print(f"OVR_logistric regression.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")

## The last two rows are not perceptron but logistic regression, it takes some time to re-run the previous cell, thus, for the sake of time saving, I will keep it as it is.

In [ ]:
# show the best hpyer parameters
print('Decison Tree:', dt_v1.best_params_)
print('Random Forest:', rf_v1.best_params_)
print('Extra Tree:', ext_tree_v1.best_params_)
print('KNN:', knn_v1.best_params_)

In [ ]:
# NN
# define the NN settings
feature_vector_length = len(X_train_reduced[0])  # assign the number of features
num_classes = len(set(Y_train))   # assign the number of the classes
print(feature_vector_length, num_classes)  # check the result

# convert the ground truth from numerical to categorical representation
y_train_cat = to_categorical(y_train_, num_classes)
y_val_cat = to_categorical(y_val, num_classes)

# set the seed
np.random.seed(123)
set_random_seed(2)

# define the NN model
model = Sequential()   # define how the 'model' looks like
model.add(Dense(input_dim=feature_vector_length, units=feature_vector_length, activation='relu'))  # input layer
model.add(Dense(num_classes, activation='softmax'))  # output layer

# configure the model and start training 
model.compile(loss='categorical_crossentropy',   # loss metric
             optimizer='sgd',   # optimizer
             metrics=['accuracy'])  # displayed metric

history = model.fit(X_train_reduced, y_train_cat, epochs=500, batch_size=32, verbose=1, validation_split=0.1, callbacks=[es])

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
# plt.ylim(0.8, 1)
plt.show()


# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# train the SVM 

svc_grid_param = {
    'C' : (0.1, 1, 10, 15, 20),
    'kernel' : ('rbf', 'linear')
}

svc_clf = SVC(random_state=123)
svc_v1 = GridSearchCV(svc_clf, svc_grid_param, n_jobs=-1, cv=5)
svc_v1.fit(X_train_reduced, y_train_)

In [ ]:
# svc
y_train_pred = svc_v1.predict(X_train_reduced)
y_val_pred = svc_v1.predict(X_val_reduced)
print(f"SVC.\tTrain:{f1_score(y_train_, y_train_pred,average = 'macro'):.4f}\tVal:{f1_score(y_val, y_val_pred,average = 'macro'):.4f}")


In [ ]:
print('SVC:', svc_v1.best_params_)

## The first four models remain similar performances, but the extra tree has a worse performance. As for the NN and SVC, We got a decent results, but both models are overfitted now, recall that without PCA, the difference between training and validation of the NN model is negligible. 